In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from lxml import etree
import requests
import time
import re
import os
from ctypes import *

In [8]:
def InitWebDriver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    
    driver.get('http://web.bookcloud.dev5.fzyun.io/#/typesetting')
    driver.implicitly_wait(3)
    
    try:
        driver.find_element_by_id('phoneInput')
        flag = True
    except:
        flag = False
    
    if flag:
        driver.find_element_by_id('phoneInput').clear()
        driver.find_element_by_id('phoneInput').send_keys('18610914536')
        driver.find_element_by_id('passwordInput').clear()
        driver.find_element_by_id('passwordInput').send_keys('q11111')
        driver.find_element_by_id('loginBtn').click()
        driver.implicitly_wait(2)
    return driver


In [9]:
def traverse(folder_path, all_files):
    fs = os.listdir(folder_path)
    for fp in fs:
        tmp_path = os.path.join(folder_path, fp)
        if not(os.path.isdir(tmp_path)):
            if re.match(r'^[~\$]', fp):
                os.remove(tmp_path)
            elif re.match(r'(.*)\.doc[x]?$', tmp_path):
                all_files.append(tmp_path)
        elif os.path.isdir(tmp_path):
            traverse(tmp_path, all_files)

In [14]:
def StructWords(path, template_name):
    all_files = []
    traverse(path, all_files)
    if len(all_files) > 0:
        driver = InitWebDriver()
        try:
            driver.find_element_by_id('headerThesisTypesetting')
            flag = True
        except:
            flag = False
        
        if flag:
            wait = WebDriverWait(driver, 300)
            for file in all_files:
                try:
                    template = driver.find_element_by_xpath("//div[@class='template-content']//li/div[@class='template-text']/div[@title='{0}']/ancestor::li".format(template_name))
                    ActionChains(driver).move_to_element(template).perform()
                    select = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='template-content']//li/div[@class='template-text']/div[@title='自动测试图书模板']/ancestor::li//button[@class='el-button hoverBtn hover-select el-button--primary']")))
                    select.click()
                    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/div/div[2]/div[1]/div/div[2]/button[1]')))
                    input_file = driver.find_element_by_xpath('//*[@id="uploadFileUpload"]/div/input')
                    time.sleep(1)
                    input_file.send_keys(file)
                    next_step = wait.until(EC.element_to_be_clickable((By.ID, "uploadSubmit")))
                    next_step.click()
                    end_step = wait.until(EC.element_to_be_clickable((By.ID, "bookNameSubmit")))
                    end_step.click()
                    new_setting = wait.until(EC.element_to_be_clickable((By.ID, "newTypesetting")))
                    new_setting.click()
                except Exception as e:
                    print(e)
                    break
            driver.quit()
   

In [15]:
StructWords(r'C:\Users\Zhao.LJ\Downloads\自动化测试',
            '自动测试图书模板')